<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Generación Aumentada por Recuperación (RAG)

En un sistema de recuperación de información (SRI), dada una consulta $q$ de un usuario se recuperan de una colección de documentos $D$ los $k$ documentos más relevantes a $q$ [1]. El lenguaje en el que se expresan los documentos y la consulta, la forma en la que se representa en una computadora la colección $D$ y la definición de relevancia varían de acuerdo con el dominio de aplicación y las necesidades del usuario. Los motores de búsqueda de Google y Yahoo, por ejemplo, son SRIs donde los documentos son páginas web y las consultas se expresan en lenguaje natural.

En el contexto de la inteligencia artificial y, particularmente, de los agentes razonadores, podemos conceptualizar a $D$ como parte de la base de conocimiento de un sistema inteligente, y a los documentos recuperados como premisas relevantes con las que se llevará a cabo un proceso de deducción para contestar $q$. Con el surgimiento de los grandes modelos de lenguaje (LLMs) se desarrollaron nuevos modelos de razonamiento subsimbólicos y neurosimbólicos que hacen uso de SRIs en el proceso de deducción [2]. Uno de estos modelos es el de generación aumentada por recuperación (RAG), el cual combina LLMs con SRIs para responder preguntas que requieren conocimiento experto [3]. ChatGPT implementa este mecanismo cuando realiza búsquedas en la web:

![Ejemplo de interacción RAG con ChatGPT](https://github.com/matizzat/InforSanLuis-2025-LLMs/blob/main/imagenes/rag_messi_chatgpt.png?raw=true)

En esa notebook implementaremos un sistema RAG que combina LLMs con SRIs y bases de datos vectoriales para responder preguntas del dominio de psicología.

## Naive RAG

El método básico de RAG parte de una colección o corpus de documentos escritos en lenguaje natural $D$, la cual contiene información factual relevante para responder preguntas de un dominio especializado. Estos documentos son considerados por el sistema como La Verdad e, idealmente, su información está bien organizada y no presenta contradicciones; es decir, no se enuncia $p$ y $\neg p$ al mismo tiempo en la colección. Los documentos pueden pertenecer a la base de conocimiento personal de un experto o pueden haber sido obtenidos desde la web mediante scraping o accediendo a bases de datos externas.

El preprocesamiento de $D$ generalmente consiste en convertir cada documento $d \in D$, que suele encontrarse en formato PDF o Markdown, a formato `.txt` para permitir su indexación en una base de datos. Una función de partición convierte a $d$ en $k$ documentos más pequeños, $d'_1, d'_2, \ldots, d'_k$, llamados chunks, cuyos tamaños son más aptos para ser analizados e interpretados por un LLM. Un desafío en esta etapa es garantizar que las distintas transformaciones de $d$ preserven su semántica original y no eliminen información relevante para su correcta interpretación. El resultado final es un nuevo corpus $D' = {d'_1, d'_2, \ldots, d'_N}$ listo para ser almacenado en una computadora.

Supongamos que $ef: \Sigma^{*} \to \mathbb{R}^{d}$ es una función de embedding, la cual transforma texto a vectores semánticos. Para cada documento $d'$ calculamos su representación vectorial $\vec{d'} = ef(d')$ y obtenemos así una colección de tuplas $\{(d'_i, \vec{d'_i})\}_{i = 1}^{N}$ que podemos almacenar en una base de datos vectorial. En un sistema de preguntas y respuestas (QA), dada una consulta $q$ del usuario, se calcula su incrustación $\vec{q} = ef(q)$ y luego se determina el conjunto $kNN(q)$ formado por los $k$ documentos más similares a $q$. Este conjunto satisface las siguientes propiedades:

* $|kNN(q)| = k.$
* Para todo $d \in D'$, se cumple que para todo $x \in kNN(x)$: $$dist(\vec{q}, \vec{x}) \leq dist(\vec{q}, \vec{d}).$$

Aquí $dist$ es una función de distancia, generalmente la distancia coseno:

$$dist(\vec{v}, \vec{u}) = 1 - \frac{\vec{v}\cdot \vec{u}}{||\vec{v}||||\vec{u}||}.$$

A partir del conjunto $kNN(q)$ y $q$ se construye un prompt $P$ con el cual solicitamos al modelo generar una respuesta a $q$. Algo que modelaríamos como:

$$answer = LLM(P).$$

Este es el modelo de RAG que utilizaremos en adelante.

## Preliminares

Instalamos la librería [ChromaDB](https://docs.trychroma.com/docs/overview/getting-started), la cual nos permitirá construir una base de datos vectorial donde almacenaremos nuestros documentos de psicología. Una vez que lo instalen, Google Colab les solicitará reiniciar la sesión:

In [ ]:
!pip install chromadb

Instalamos una librería que nos permitirá visualizar nuestra base de datos vectorial y ver como están distribuidos nuestros documentos en el espacio:

In [ ]:
!pip install umap-learn[plot]

Importamos recursos importantes desde Github:

In [ ]:
!git clone https://github.com/matizzat/InforSanLuis-2025-LLMs
%cd InforSanLuis-2025-LLMs

fatal: destination path 'InforSanLuis-2025-LLMs' already exists and is not an empty directory.
/content/InforSanLuis-2025-LLMs


Instanciamos nuestro cliente Genai para acceder al LLM Gemini:

In [ ]:
from google.colab import userdata
from google.genai import types
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai_client = genai.Client(api_key=GOOGLE_API_KEY)

## Creación y visualización de la base de datos semántica

En esta sección crearemos nuestra base de datos semántica con ayuda de la librería ChromaDB. Nuestro lote de documentos es del dominio de psicología y fueron tomador del diccionario [APA](https://dictionary.apa.org/).

Importamos librerías necesarias:

In [ ]:
from sentence_transformers import SentenceTransformer
from functools import partial
import pandas as pd
import numpy as np
import csv as csv
import umap.plot
import chromadb
import pprint
import umap
import json

Importamos a un modelo **Sentence Transformer**, el cual tiene como función principal convertir un texto $w$ en un vector numérico y que representa implícitamente la semántica de $w$. Los modelos transformadores de sentencias se construyen a partir de los LLMs codificadores tales como BERT y RoBERTa:

In [ ]:
sentence_transformer =  SentenceTransformer("all-MiniLM-L6-v2")

Abrimos el lote de documentos de psicología. Cada documento posee un título y un texto asociado. En total hay 107 documentos y nosotros imprimimos los primeros 5:

In [ ]:
with open('./datos/psychology.json', 'r') as f:
    chunks = json.load(f)
pprint.pprint(chunks[0:5])

[{'text': 'Mental health is a state of mind characterized by emotional '
          'well-being, good behavioral adjustment, relative freedom from '
          'anxiety and disabling symptoms, and a capacity to establish '
          'constructive relationships and cope with the ordinary demands and '
          'stresses of life.\n',
  'title': 'mental_health'},
 {'text': 'Flourishing is a condition denoting good mental and physical '
          'health: the state of being free from illness and distress but, more '
          'important, of being filled with vitality and functioning well in '
          'one’s personal and social life.\n',
  'title': 'flourishing'},
 {'text': 'Languishing is the condition of absence of mental health, '
          'characterized by ennui, apathy, listlessness, and loss of interest '
          'in life.\n',
  'title': 'languishing'},
 {'text': 'Depression is a negative affective state, ranging from unhappiness '
          'and discontent to an extreme feeling o

Convertimos cada uno de los textos de nuestro lote de datos en un vector numérico llamado **embedding** mediante el transformador de sentencias:

In [ ]:
embeddings = sentence_transformer.encode([chunk['text'] for chunk in chunks])

### Almacenamos vectores con ChromaDB

[ChromaDB](https://www.trychroma.com/) es un sistema de gestión de bases de datos vectoriales (VDBMS) nativo que utilizaremos para almacenar los embeddings de nuestro cuerpo de documentos. Fue diseñado con el objetivo de desarrollar aplicaciones que requieren de búsquedas semánticas sobre una gran colección de datos no estructurados, tales como textos, imágenes o audios. Es por ello que lo usaremos como soporte para construir nuestro sistema RAG.

#### Modelo de datos

El modelo de datos de ChromaDB tiene las siguientes componentes:
* **Item:** Un item es una unidad de almacenamiento que puede modelarse como una tupla $(id, w, \vec{w}, met)$ donde $id$ es un identificador alfanumérico, $w$ es un documento, $\vec{w}$ es el embedding de $w$ y $met$ son metadatos del documento. En python $met$ se especifica como un diccionario.

* **Colección:** Una colección es un conjunto de uno o más items identificado univocamente por un nombre y donde todos sus incrustaciones tienen la misma dimensión $d$. La colección puede estar asociada a una función de incrustación $ef$ que se invoca automáticamente al realizar operaciones de alta o búsqueda por similitud. ChromaDB construye un índice HNSW por cada colección para resolver consultas de búsquedas aproximadas de vecinos más cercanos (ANN). Al igual que los items, las colecciones pueden tener asociados metadatos para facilitar su interpretación.

* **Base de Datos**: Una base de datos es un conjunto de una o más colecciones. El nombre de cada colección de la base de datos es único.

Ahora utilizaremos ChromaDB para crear el directorio donde almacenaremos nuestros embeddings. Primero instaciamos un cliente y especificamos donde queremos crear la base semántica:

In [ ]:
cli = chromadb.PersistentClient(path="./datos/base_semantica")

Vamos a crear una colección `psychology` y dejaremos sin especificar la función de embedding. Esto implica que al momento de agregar datos y realizar búsquedas nosotros deberemos calcular previamente las incrustaciones de los documentos. También especificamos los siguientes parámetros que afectan la construcción del índice HNSW:

* `space`: Es la función de distancia, que en nuestro caso es la distancia coseno.

* `ef_construction`: A valores más altos tenemos búsquedas más precisas pero tiempos de construcción más lentos.

*  `ef_search`: A valores más altos tenemos búsquedas más precisas pero tiempos de búsqueda más lentos.

Nuestra configuración de los parámetros `ef_construction` y `ef_search` prioriza exactitud en las búsquedas sobre tiempos de búsquedas rápidos y velocidad de construcción del índice.

In [ ]:
col = cli.create_collection(
    name="psychology",
    embedding_function=None,
    configuration={
        "hnsw": {
            "space": "cosine",
            "ef_construction": 1000,
            "ef_search": 200
        }
    },
    metadata = {
        "description": "Semantic database for psychology documents."
    }
)

Agregamos los documentos a nuestra colección:

In [ ]:
col.add(
    ids = [chunk['title'] for chunk in chunks],
    embeddings = embeddings,
    documents = [chunk['text'] for chunk in chunks]
)

### Visualizamos la base de datos semántica con UMAP

[UMAP](https://umap-learn.readthedocs.io/en/latest/) es un algoritmo de reducción de dimensión utilizado para visualizar y preprocesar lotes de datos en aplicaciones de aprendizaje automático. Nosotros lo usaremos para graficar las embeddings de documentos generados por el transformador de sentencias.

Ejecuten la siguiente celda de código y observen cómo se distribuyen los documentos en el espacio vectorial. Si pasan el cursor sobre un punto se mostrará su respectivo texto. Fijense que documentos semánticamente similares, como aquellas que tratan sobre la anatomía del cerebro, forman pequeños grupos o clusters. Cuando ustedes realizan una consulta a un sistema SRI esta también es transformada a un vector para poder compararse con los demás documentos:

In [ ]:
umap.plot.output_notebook()

hover_data = pd.DataFrame({'title': [chunk['title'] for chunk in chunks],
                           'text': [chunk['text'] for chunk in chunks]})

U = umap.UMAP(n_components = 2, n_neighbors = 15, min_dist = 0.1, metric = 'cosine', densmap = True)
U.fit(embeddings)
p = umap.plot.interactive(U, hover_data=hover_data, point_size=10)
umap.plot.show(p)

### La función de recuperación

A continuación definimos una función que recibe una consulta $q$ y un número $k$, calcula el embedding $\vec{q}$ y computa el conjunto $kNN(q)$ la cual contiene los $k$ documentos más relevantes a $q$:

In [ ]:
def retrieve_knn(query: str, k: int) -> list[str]:
  query_embedding = sentence_transformer.encode(query)
  knn = col.query(query_embeddings = [query_embedding], n_results = k)
  return knn['documents'][0]


Ejecuten la siguiente celda de código para observar los tres documentos recuperados para la consulta *"What are the different parts of the brain?"* Les recomiendo probar con otras preguntas y números de vecinos:

In [ ]:
query_result = retrieve_knn("What are the different parts of the brain?", 3)

for i in range(len(query_result)):
  print('Doc ' + str(i))
  print(query_result[i])


Doc 0
The brain is the enlarged, anterior part of the central nervous system within the skull.
The young adult human brain weighs about 1,450 g, and its outer layer (the cerebral cortex) contains over 10 billion nerve cells.
The brain develops by differentiation of the embryonic neural tube along an anterior–posterior axis to form three main regions—the forebrain, midbrain, and hindbrain—that can be further subdivided on the basis of anatomical and functional criteria.
The cortical tissue is concentrated in the forebrain, and the midbrain and hindbrain structures are often considered together as the brainstem.
It is also called encephalon.

Doc 1
The cerebrum is the largest part of the brain, forming most of the forebrain and lying in front of and above the cerebellum.
It consists of two cerebral hemispheres bridged by the corpus callosum.
Each hemisphere is divided into four main lobes: the frontal lobe, occipital lobe, parietal lobe, and temporal lobe.
The outer layer of the cerebrum

# Tarea

La tarea de ustedes será terminar el proceso del RAG implementando el último paso, el cual consiste en invocar a un LLM generativo a que responda una consulta en función de los textos que se encuentran en la base de datos semántica. Para ello deberán realizar lo siguiente:

1. Escribir un prompt de sistema y una plantilla de prompt de usuario que solicite a Gemini que responda una consulta del dominio de psicología. Tengan en cuenta que:

   * En el prompt de sistema ustedes especifican al modelo el rol que cumple, le dan una breve descripción de la tarea a resolver y, opcionalmente, le muestran ejemplos de cómo se resuelve la tarea para preguntas concretas.

    * En el prompt de usuario ustedes presentan la consulta concreta y muestran los documentos específicos que recuperaron de la base de datos semántica para responder la consulta.

    * Pueden guardar las instrucciones en un archivo .txt o dejarlas como constantes string en una celda de código.

2. Escribir una función answer_query la cual recibe como entrada una consulta $q$ y devuelve la respuesta del modelo junto a la lista de documentos recuperados para responder la consulta. Esta función debe recuperar los 3 (tres) documentos más relevantes a $q$, formatear el prompt de usuario e invocar al modelo Gemini para que responda la consulta.

Muestren en la notebook las respuestas del LLM y los documentos recuperados para las siguientes preguntas:

* What are the different parts of the brain?

* What are the characteristics of extroversion?

* How old is Lionel Messi?

* What are dinosaurs?

* What are the symptoms of depression?

Pueden utilizar cualquiera de las funciones definidas en esta notebook.

**Consejos**:

* Pueden especificarle al modelo que responda *"No poseo información suficiente"* cuando la consulta no pueda responderse en función de los documentos recuperados.

* Pueden especificarle al modelo que, al momento de contestar una consulta, explicite las frases de los documentos recuperados que utiliza para generar su respuesta.

---

### Entrega

Cuando finalicen, suban su notebook y todos los archivos generados a un repositorio público de GitHub. Envíen el enlace del repositorio al correo mat.torreta@gmail.com con el asunto:

* InforSanLuis25-LLMs Entrega

En el cuerpo del correo deben incluir:

* Nombre completo

* DNI

La entrega deberá realizarse antes del viernes 05 de diciembre a las 23:59 para aprobar el curso.

---

### Solución

Pueden escribir las instrucciones en la siguiente celda de código o guardarlas en un archivo .txt si se les hace más cómodo.

In [ ]:
system_prompt = """You are a ..."""
user_prompt = """Passages: ... Query: ... Answer: ..."""

Pueden utilizar esta función para invocar al modelo Gemini:

In [ ]:
def invoke_llm(system: str, user: str):
    """
    Invokes the Gemini 2.5 Flash language model.

    Parameters:
        system (str): System instructions for the model.
        user   (str): User query or instructions.

    Returns:
        str: Text generated by the model.
    """
    global genai_client

    response = genai_client.models.generate_content(
        config = types.GenerateContentConfig(
            system_instruction = system,
            temperature = 0.1
        ),
        model = "gemini-2.5-flash",
        contents = user
    )

    return response.text

Completen la siguiente porción de código para responder las consultas:

In [ ]:
def answer_query(query: str) -> tuple[str, list[str]]:
    """
    IMPORTANTE:

    Aquí deben implementar la estrategia para responder
    consultas utilizando los documentos de la base de
    datos semántica.
    """

    raise RuntimeError("Función no implementada.")

# Referencias

1. *Modern Information Retrieval* de Ricarzo Baeza-Yates.
2. [AI Reasoning in Deep Learning Era: From Symbolic AI to Neural–Symbolic AI](https://www.mdpi.com/2227-7390/13/11/1707) de Baoyu Liang, et al.
3. [Retrieval-Augmented Generation for Large Language Models: A Survey](https://arxiv.org/abs/2312.10997) de Yunfan Gao, et al.